## Coulomb Potential
    IX. Write a function to calculate the Coulomb energy and potential, then plot the potential for the previous density.

Electrostatic energy or Hatree energy(in 1D)
$$ E_{Ha}=\frac{1}{2}\iint \frac{n(x)n(x')}{\sqrt{(x-x')^2+\varepsilon}}dxdx'$$

where $\varepsilon$ is a small positive constant

The potential is given by:

$$ v_{Ha}=\int \frac{n(x')}{\sqrt{(x-x')^2+\varepsilon}}dx'$$

In [1]:
from scipy.integrate import quad
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.integrate import simps

In [2]:
def D(x, N=100):
    '''
    Returns the operator form of first-derivative

            Parameters:
                    x (array): grid of 1D array
                    N (int): number of points on the grid

            Returns:
                    D (array): first-derivative operator
    '''
    h = x[1] - x[0]
    k = [np.ones(N-1), -np.ones(N-1)]
    offset = [1, -1]
    D = diags(k, offset).toarray()
    D = D/(2 * h)
    # # Boundary values where it is not well defined
    D[0, 0] = 0
    D[0, 1] = 0
    D[1, 0] = 0
    D[N-1, N-2] = 0
    D[N-2, N-1] = 0
    D[N-1, N-1] = 0
    return D


def D2(x, N=100):
    '''
    Returns the operator form of second-derivative

            Parameters:
                    x (array): grid of 1D array
                    N (int): number of points on the grid

            Returns:
                    D (array): first-derivative operator
    '''
    h = x[1] - x[0]
    k1 = [np.ones(N-1), -2*np.ones(N), np.ones(N-1)]
    offset = [-1, 0, 1]
    D2 = diags(k1, offset).toarray()
    D2 = D2/(h ** 2)
    # Boundary values where it is not well defined
    D2[0, 0] = 0
    D2[0, 1] = 0
    D2[1, 0] = 0
    D2[N-1, N-2] = 0
    D2[N-2, N-1] = 0
    D2[N-1, N-1] = 0
    return D2


def normalise_psi(psi, x):
    '''
    Normalises the given wavefunction

            Parameters:
                    psi (array): wavefunction psi
                    x (array): 1D grid of array

            Returns:
                    normalised psi
    '''
    int_psi_square = simps(abs(psi) ** 2, x)
    return psi/np.sqrt(int_psi_square)


def get_occupation_num(nElectron, maxElectron=2):
    '''
    Returns a list of occupation numbers for a given number of electrons

            Parameters:
                    nElectron (int): number of electrons
                    maxElectron (int): max number of allowed elecrons in one state

            Returns:
                    fn (array): occupation number
    '''
    nFloor = np.floor_divide(nElectron, 2)
    fn = maxElectron * np.ones(nFloor)
    if nElectron % 2:
        fn = np.append(fn, 1)
    return fn


def get_density(nElectron, psi, x):
    '''
    Returns electron density for a given number of electrona and wavefunction

            Parameters:
                    nElectron (int): number of electrons
                    psi (array): wavefunction
                    x (array): 1D grid of array

            Returns:
                    eDensity (array): electron density
    '''
    psiNorm = normalise_psi(psi, x)  # Normalisation
    fn = get_occupation_num(nElectron)
    eDensity = np.zeros(N)
    for f_n, psi in zip(fn, psiNorm.T):
        eDensity += f_n * (psi ** 2)
    return eDensity


def integrate(x, y):
    '''
    Returns the integration by simpson's method

            Parameters:
                    x (array): 1D grid of arra
                    y (array): integrand

            Returns:
                    result (float): result of the integration
    '''
    result = simps(y, x)
    return result


def calculate_exchange(eDensity, x):
    '''
    Returns exchange energy and potential for a given electron density

            Parameters:
                    eDensity (array): electron density
                    x (array): 1D grid of array

            Returns:
                    energy (float): exchange energy
                    potential (float): exchange potential
    '''
    energy = -3/4 * (3/np.pi) ** (1/3) * integrate(x, eDensity ** (4/3))
    potential = -(3/np.pi) ** (1/3) * eDensity ** (1/3)
    return energy, potential


def calculate_coulomb(eDensity, x, eps=0.1):
    '''
    Returns Coulomb/Hartree energy and potential for a given electron density

            Parameters:
                    eDensity (array): electron density
                    x (array): 1D grid of array

            Returns:
                    energy (float): Hartee energy
                    potential (float): Hartree potential
    '''
    h = x[1] - x[0]
    nx = eDensity[np.newaxis, :]
    nxp = eDensity[:, np.newaxis]
    w = x[np.newaxis, :]
    wp = x[:, np.newaxis]
    energy = 0.5 * np.sum(nx * nxp * h ** 2/np.sqrt((w - wp) ** 2 + eps))
    potential = np.sum((nx * h)/np.sqrt((w - wp) ** 2 + eps), axis=-1)
    return energy, potential

In [3]:
L = 5
N = 200
x = np.linspace(-L, L, N)
X = np.diagflat(x*x)
H = -D2(x, N)/2 + X
E, V = np.linalg.eigh(H)
psi = V
nElectron = 6

In [4]:
density = get_density(nElectron, psi, x)

In [5]:
Ec, Vc = calculate_coulomb(density, x)

In [6]:
Ec

22.246896534544184

In [7]:
Vc

array([1.25405611, 1.26798576, 1.28224368, 1.29684221, 1.31179429,
       1.32711361, 1.34281459, 1.35891245, 1.37542327, 1.39236408,
       1.40975288, 1.42760878, 1.44595203, 1.46480415, 1.48418803,
       1.50412803, 1.52465014, 1.54578207, 1.56755347, 1.58999607,
       1.61314391, 1.63703351, 1.66170417, 1.68719824, 1.71356143,
       1.74084316, 1.76909699, 1.79838103, 1.82875851, 1.86029832,
       1.89307566, 1.92717281, 1.96267989, 1.99969581, 2.03832927,
       2.07869983, 2.12093914, 2.16519218, 2.21161861, 2.26039416,
       2.31171195, 2.36578384, 2.4228416 , 2.4831378 , 2.54694646,
       2.61456307, 2.6863041 , 2.76250552, 2.84352032, 2.92971481,
       3.02146335, 3.11914141, 3.22311678, 3.33373883, 3.45132572,
       3.57614967, 3.70842057, 3.84826812, 3.99572318, 4.15069904,
       4.31297333, 4.48217193, 4.65775582, 4.83901228, 5.02505167,
       5.21481103, 5.40706548, 5.60044808, 5.79347857, 5.98460062,
       6.17222687, 6.35479027, 6.53079958, 6.69889647, 6.85791